Preparation of contact pattern matrix and $$\beta_{home}$$

# Transforming The Above Files:
### One should choose the division set for the model via the variable <b>'cell_name'</b> in the <i>indices.py</i> file in the model. The options are :
* 20
* 250
* 870

### In order to run this notebook One should make sure the above files exist:
* Data/division_choice
    * Data/division_choice/'cell_name'/cell2name.xlsx
    * Data/division_choice/'cell_name'/mat_macro_model_df.pickle
    * Data/division_choice/'cell_name'/taz2cell.xlsx
* Data/raw
    * Data/raw/pop2taz.xlsx
    * Data/raw/religion2taz.csv

    * Data/raw/leisure_mtx.csv
    * Data/raw/work_mtx.csv
    * Data/raw/school_mtx.csv

    * Data/raw/secularism_base_home.xlsx
    * Data/raw/haredi_base_home.xlsx
    * Data/raw/arabs_base_home.xlsx

    * Data/raw/stay_home_taz/no_school.csv      =======MISSING=======
    * Data/raw/stay_home_taz/no_work.csv        =======MISSING=======
    * Data/raw/stay_home_taz/routine.csv        =======MISSING=======

### This Notebook generates the above files:
* Data/demograph
    - Data/demograph/age_dist_area.csv
    - Data/demograph/religion_dis.csv
    - Data/demograph/beta_home.pickle
* Data/base_contact_mtx
    - Data/base_contact_mtx/full_home_mtx.csv
    - Data/base_contact_mtx/full_leisure_mtx_no_school.csv
    - Data/base_contact_mtx/full_leisure_mtx_no_work.csv
    - Data/base_contact_mtx/full_leisure_mtx_routine.csv
    - Data/base_contact_mtx/full_work_mtx_no_school.csv
    - Data/base_contact_mtx/full_work_mtx_no_work.csv
    - Data/base_contact_mtx/full_work_mtx_routine.csv
* Data/stay home
    - Data/stay home/no_school.csv              ========TODO=========
    - Data/stay home/no_work.csv                ========TODO=========
    - Data/stay home/routine.csv                ========TODO=========

# Imports

In [1]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
from scipy import optimize
import sys
import os
sys.path.append('../SEIR_full/')
sys.path.append('..')
from SEIR_full import *
%matplotlib inline

# Data Creation

## Consts

In [54]:
cell_name = '20'

## Creating demograph/age_dist_area.csv

In [56]:
pop_dist = pd.read_excel('../Data/raw/pop2taz.xlsx', header=2)

In [57]:
pop_dist.columns

Index(['אזור 2630', ' מודל מקור', 'תכולת אזורי תנועה במודל המקור',
       'כלול באזור תנועה (1250) ', 'סמל ישוב', 'שם היישוב/ ים',
       'אזור סטטיסטי ביישוב', 'פירוט האזור', 'שם ישוב, שכונה, מיתקן או רחובות',
       'שימוש קרקע עיקרי', 'ערכי שימושי קרקע', 'אוכלוסייה', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'גילאים', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'מקושר ל 1250', 'מקושר ל 250',
       'מקושר ל 33', 'נפה', 'מחוז', 'שטח (מ"ר)'],
      dtype='object')

In [58]:
ages_list = ['Unnamed: '+ str(i) for i in range(17,32)]

pop_dist = pop_dist[['אזור 2630', 'גילאים'] + ages_list]
pop_dist.columns = ['id'] + list(pop_dist.iloc[0,1:])

In [59]:
pop_dist = pop_dist.drop([0, 2631, 2632, 2633])

In [60]:
pop_dist['tot_pop'] = pop_dist.iloc[:, 1:].sum(axis=1)
pop_dist['tot_pop'][pop_dist['tot_pop'] == 0] = 1
pop_dist = pop_dist.iloc[:,1:-1].div(pop_dist['tot_pop'], axis=0).join(pop_dist['id']).join(pop_dist['tot_pop'])
pop_dist['tot_pop'][pop_dist['tot_pop'] == 1] = 0
pop_dist['tot_pop'] = pop_dist['tot_pop']/pop_dist['tot_pop'].sum()
pop_dist.iloc[:,:-2] = pop_dist.iloc[:,:-2].mul(pop_dist['tot_pop'], axis=0)
pop_dist

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75+,id,tot_pop
1,4.46203e-05,4.73732e-05,5.3682e-05,8.167e-05,4.66849e-05,3.5788e-05,1.50264e-05,1.58293e-05,2.14498e-05,2.13351e-05,2.54645e-05,2.25969e-05,1.51411e-05,5.27643e-06,2.40881e-06,5.39114e-06,1,0.000460
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.000000
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.000000
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0.000000
5,1.23881e-05,1.05529e-05,7.57053e-06,6.19407e-06,4.5882e-06,6.19407e-06,7.91465e-06,1.05529e-05,9.97934e-06,8.60288e-06,6.19407e-06,7.34112e-06,8.71758e-06,1.04382e-05,9.29111e-06,8.14406e-06,5,0.000135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,0.000239275,0.000199472,0.00016724,0.000167699,0.000135008,0.000105185,7.30671e-05,5.9876e-05,5.59761e-05,4.59967e-05,3.06262e-05,1.92704e-05,1.60587e-05,1.06676e-05,8.25876e-06,8.14406e-06,2626,0.001342
2627,0.000137187,0.00012262,0.000113443,0.000110576,8.78641e-05,6.80201e-05,4.64555e-05,4.46203e-05,3.93438e-05,3.48703e-05,1.88116e-05,1.3994e-05,9.1764e-06,5.16173e-06,5.27643e-06,7.68524e-06,2627,0.000865
2628,0.000279536,0.000242945,0.000225625,0.000214154,0.000178252,0.000135925,9.81875e-05,8.06376e-05,7.67377e-05,6.21701e-05,4.06056e-05,2.62675e-05,1.97293e-05,1.23881e-05,1.07823e-05,1.04382e-05,2628,0.001714
2629,0.000222184,0.000217825,0.000191787,0.000182381,0.000193163,0.000192819,0.00019821,0.000188919,0.00019236,0.000187199,0.000162422,0.000129731,0.000114705,8.87817e-05,6.49231e-05,6.53819e-05,2629,0.002593


In [61]:
taz2cell = pd.read_excel('../Data/division_choice/'+ cell_name + '/taz2cell.xlsx')

In [62]:
taz2cell = taz2cell[['taz_id', 'cell_id']]
taz2cell.columns = ['id', 'new_id']

In [63]:
taz2cell

,id,new_id
0,1,11
1,2,11
2,3,11
3,4,11
4,5,11
...,...,...
2625,2626,71
2626,2627,71
2627,2628,71
2628,2629,71


In [64]:
pop_cell = pop_dist.merge(taz2cell, left_on='id', right_on='id')
pop_cell['new_id'] = pop_cell['new_id'].astype(str)
pop_cell.sort_values(by='new_id')

,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75+,id,tot_pop,new_id
0,4.46203e-05,4.73732e-05,5.3682e-05,8.167e-05,4.66849e-05,3.5788e-05,1.50264e-05,1.58293e-05,2.14498e-05,2.13351e-05,2.54645e-05,2.25969e-05,1.51411e-05,5.27643e-06,2.40881e-06,5.39114e-06,1,0.000460,11
93,2.93645e-05,2.83321e-05,2.52351e-05,2.37439e-05,2.24822e-05,2.50057e-05,2.71851e-05,2.94792e-05,2.54645e-05,2.39734e-05,1.97293e-05,1.89263e-05,1.91557e-05,2.19087e-05,1.45675e-05,2.54645e-05,94,0.000380,11
92,3.81968e-05,3.60174e-05,3.20027e-05,3.01674e-05,3.32645e-05,3.87703e-05,3.48703e-05,4.03762e-05,3.25762e-05,3.10851e-05,2.6841e-05,2.50057e-05,2.67263e-05,2.54645e-05,2.01881e-05,3.36086e-05,93,0.000505,11
91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92,0.000000,11
90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,91,0.000000,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2441,0.000000,71
2439,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2440,0.000000,71
2438,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2439,0.000000,71
2444,0.000187428,0.000164831,0.000159669,0.000147969,0.000111608,8.08671e-05,5.72378e-05,5.45996e-05,5.19614e-05,3.78527e-05,2.52351e-05,2.00734e-05,1.58293e-05,6.65289e-06,7.22642e-06,6.07937e-06,2445,0.001135,71


In [65]:
def make_pop(df):
    df = df.iloc[:,0:-2]
    return df.sum(axis=0)

In [66]:
pop_cell = pop_cell.groupby(by='new_id').apply(lambda df: make_pop(df))

In [67]:
pop_cell

,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75+,id
new_id,,,,,,,,,,,,,,,,,
11,0.003902,0.003036,0.002482,0.002181,0.002552,0.004585,0.005781,0.004755,0.003690,0.002942,0.002344,0.002287,0.002327,0.002315,0.001860,0.003458,8128.0
11_betshemesh,0.008565,0.007406,0.006251,0.005643,0.005244,0.005363,0.005882,0.005673,0.005186,0.004490,0.003645,0.003531,0.003668,0.003761,0.002961,0.005319,35828.0
21,0.007581,0.007176,0.006258,0.005601,0.005079,0.004949,0.005576,0.005983,0.006092,0.005317,0.004273,0.004175,0.004335,0.004426,0.003292,0.005431,77322.0
22,0.001892,0.001742,0.001623,0.001600,0.001551,0.001550,0.001520,0.001496,0.001592,0.001460,0.001254,0.001270,0.001261,0.001242,0.000862,0.001313,29697.0
23,0.002338,0.002074,0.001701,0.001463,0.001464,0.001580,0.001750,0.001754,0.001743,0.001423,0.001180,0.001207,0.001375,0.001475,0.001068,0.001583,28725.0
24,0.003750,0.003418,0.002907,0.002664,0.002604,0.002619,0.002779,0.002852,0.002846,0.002386,0.002050,0.002175,0.002338,0.002434,0.001748,0.002972,64950.0
29,0.005077,0.005115,0.004848,0.004281,0.003709,0.003085,0.003073,0.003366,0.003712,0.003467,0.002921,0.002654,0.002335,0.002040,0.001415,0.002489,85428.0
31,0.003006,0.003219,0.003186,0.003099,0.002482,0.002057,0.001842,0.002016,0.002360,0.002293,0.001863,0.001528,0.001296,0.001084,0.000762,0.001143,81700.0
32,0.008057,0.008017,0.007279,0.006424,0.005297,0.004654,0.004334,0.004468,0.004318,0.003668,0.002994,0.002580,0.002164,0.001931,0.001421,0.002228,192672.0


In [68]:
A

{0: '0-4',
 1: '5-9',
 2: '10-19',
 3: '20-29',
 4: '30-39',
 5: '40-49',
 6: '50-59',
 7: '60-69',
 8: '70+'}

In [69]:
pop_cell['10-19'] = pop_cell['10-14'] + pop_cell['15-19']
pop_cell['20-29'] = pop_cell['20-24'] + pop_cell['25-29']
pop_cell['30-39'] = pop_cell['30-34'] + pop_cell['35-39']
pop_cell['40-49'] = pop_cell['40-44'] + pop_cell['45-49']
pop_cell['50-59'] = pop_cell['50-54'] + pop_cell['55-59']
pop_cell['60-69'] = pop_cell['60-64'] + pop_cell['65-69']
pop_cell['70+'] = pop_cell['70-74'] + pop_cell['75+']
pop_cell = pop_cell[list(A.values())]
pop_cell

,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
new_id,,,,,,,,,
11,0.003902,0.003036,0.004663,0.007137,0.010536,0.006631,0.004630,0.004642,0.005318
11_betshemesh,0.008565,0.007406,0.011894,0.010607,0.011555,0.009676,0.007175,0.007429,0.008280
21,0.007581,0.007176,0.011858,0.010028,0.011560,0.011409,0.008448,0.008761,0.008723
22,0.001892,0.001742,0.003223,0.003102,0.003015,0.003052,0.002524,0.002503,0.002174
23,0.002338,0.002074,0.003164,0.003044,0.003504,0.003166,0.002388,0.002851,0.002650
24,0.003750,0.003418,0.005571,0.005223,0.005631,0.005232,0.004225,0.004772,0.004720
29,0.005077,0.005115,0.009129,0.006794,0.006439,0.007179,0.005575,0.004375,0.003904
31,0.003006,0.003219,0.006285,0.004540,0.003858,0.004654,0.003392,0.002380,0.001905
32,0.008057,0.008017,0.013703,0.009951,0.008802,0.007986,0.005574,0.004095,0.003650


In [70]:
pop_cell = pop_cell / pop_cell.sum().sum()

In [71]:
pop_cell.reset_index(inplace=True)
pop_cell.columns = ['cell_id'] + list(A.values())

In [72]:
pop_cell

,cell_id,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
0,11,0.003902,0.003036,0.004663,0.007137,0.010536,0.006631,0.004630,0.004642,0.005318
1,11_betshemesh,0.008565,0.007406,0.011894,0.010607,0.011555,0.009676,0.007175,0.007429,0.008280
2,21,0.007581,0.007176,0.011858,0.010028,0.011560,0.011409,0.008448,0.008761,0.008723
3,22,0.001892,0.001742,0.003223,0.003102,0.003015,0.003052,0.002524,0.002503,0.002174
4,23,0.002338,0.002074,0.003164,0.003044,0.003504,0.003166,0.002388,0.002851,0.002650
5,24,0.003750,0.003418,0.005571,0.005223,0.005631,0.005232,0.004225,0.004772,0.004720
6,29,0.005077,0.005115,0.009129,0.006794,0.006439,0.007179,0.005575,0.004375,0.003904
7,31,0.003006,0.003219,0.006285,0.004540,0.003858,0.004654,0.003392,0.002380,0.001905
8,32,0.008057,0.008017,0.013703,0.009951,0.008802,0.007986,0.005574,0.004095,0.003650
9,41,0.007217,0.006297,0.011206,0.008920,0.006004,0.004405,0.003556,0.002967,0.003079


In [73]:
pop_cell.to_csv('../Data/demograph/age_dist_area.csv')

## Creating demograph/religion_dis.csv

In [83]:
religion2taz = pd.read_csv('../Data/raw/religion2taz.csv')
religion2taz.sort_values(by='taz_id', inplace=True)
religion2taz.columns = ['id', 'Orthodox', 'Druze', 'Other', 'Sacular', 'Muslim', 'Christian']
religion2taz['Jewish'] = religion2taz['Orthodox'] + religion2taz['Sacular']
religion2taz

,id,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish
0,1,0.029482,0.0,0.0,0.970310,0.000207,0.0,0.999793
1,2,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
2,3,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,4,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
4,5,0.010413,0.0,0.0,0.915182,0.074406,0.0,0.925594
...,...,...,...,...,...,...,...,...
2545,2626,0.000000,0.0,0.0,0.000303,0.999697,0.0,0.000303
2546,2627,0.000000,0.0,0.0,0.002635,0.997365,0.0,0.002635
2547,2628,0.000000,0.0,0.0,0.002553,0.997447,0.0,0.002553
2548,2629,0.001288,0.0,0.0,0.940031,0.058681,0.0,0.941319


In [84]:
taz2cell = pd.read_excel('../Data/division_choice/'+ cell_name + '/taz2cell.xlsx')

In [85]:
taz2cell = taz2cell[['taz_id', 'cell_id']]
taz2cell.columns = ['id', 'new_id']
taz2cell

,id,new_id
0,1,11
1,2,11
2,3,11
3,4,11
4,5,11
...,...,...
2625,2626,71
2626,2627,71
2627,2628,71
2628,2629,71


In [86]:
religion2taz = religion2taz.merge(taz2cell, on='id')
religion2taz['new_id'] = religion2taz['new_id'].astype(str)
religion2taz.sort_values(by='new_id', inplace=True)
religion2taz

,id,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish,new_id
0,1,0.029482,0.0,0.0,0.970310,0.000207,0.0,0.999793,11
93,94,0.008043,0.0,0.0,0.991400,0.000557,0.0,0.999443,11
92,93,0.000066,0.0,0.0,0.957130,0.042804,0.0,0.957196,11
91,92,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11
90,91,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11
...,...,...,...,...,...,...,...,...,...
2440,2441,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,71
2439,2440,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,71
2438,2439,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,71
2444,2445,0.000037,0.0,0.0,0.027196,0.972767,0.0,0.027233,71


In [87]:
pop_dist = pd.read_excel('../Data/raw/pop2taz.xlsx', header=2)
ages_list = ['Unnamed: '+ str(i) for i in range(17,32)]

pop_dist = pop_dist[['אזור 2630', 'גילאים'] + ages_list]
pop_dist.columns = ['id'] + list(pop_dist.iloc[0,1:])
pop_dist = pop_dist.drop([0, 2631, 2632, 2633])
pop_dist['tot_pop'] = pop_dist.iloc[:, 1:].sum(axis=1)
pop_dist['tot_pop'][pop_dist['tot_pop'] == 0] = 1
pop_dist = pop_dist.iloc[:,1:-1].div(pop_dist['tot_pop'], axis=0).join(pop_dist['id']).join(pop_dist['tot_pop'])
pop_dist['tot_pop'][pop_dist['tot_pop'] == 1] = 0
pop_dist['tot_pop'] = pop_dist['tot_pop']/pop_dist['tot_pop'].sum()
pop_dist.iloc[:,:-2] = pop_dist.iloc[:,:-2].mul(pop_dist['tot_pop'], axis=0)
pop_dist = pop_dist[['id', 'tot_pop']]
pop_dist

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,id,tot_pop
1,1,0.000460
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000135
...,...,...
2626,2626,0.001342
2627,2627,0.000865
2628,2628,0.001714
2629,2629,0.002593


In [88]:
religion2taz = religion2taz.merge(pop_dist, on='id')
religion2taz.sort_values(by='id', inplace=True)
religion2taz

,id,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish,new_id,tot_pop
0,1,0.029482,0.0,0.0,0.970310,0.000207,0.0,0.999793,11,0.000460
91,2,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11,0.000000
90,3,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11,0.000000
89,4,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11,0.000000
88,5,0.010413,0.0,0.0,0.915182,0.074406,0.0,0.925594,11,0.000135
...,...,...,...,...,...,...,...,...,...,...
2440,2626,0.000000,0.0,0.0,0.000303,0.999697,0.0,0.000303,71,0.001342
2439,2627,0.000000,0.0,0.0,0.002635,0.997365,0.0,0.002635,71,0.000865
2438,2628,0.000000,0.0,0.0,0.002553,0.997447,0.0,0.002553,71,0.001714
2527,2629,0.001288,0.0,0.0,0.940031,0.058681,0.0,0.941319,71,0.002593


In [89]:
religion2taz.sort_values(by='new_id').head(20)

,id,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish,new_id,tot_pop
0,1,0.029482,0.0,0.0,0.970310,0.000207,0.0,0.999793,11,0.000460
1,94,0.008043,0.0,0.0,0.991400,0.000557,0.0,0.999443,11,0.000380
2,93,0.000066,0.0,0.0,0.957130,0.042804,0.0,0.957196,11,0.000505
3,92,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11,0.000000
4,91,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,11,0.000000
5,90,0.011012,0.0,0.0,0.988988,0.000000,0.0,1.000000,11,0.000749
6,89,0.022895,0.0,0.0,0.976817,0.000288,0.0,0.999712,11,0.000416
7,88,0.010394,0.0,0.0,0.914542,0.075064,0.0,0.924936,11,0.000320
8,87,0.013906,0.0,0.0,0.982596,0.003498,0.0,0.996502,11,0.000413
9,86,0.005737,0.0,0.0,0.431398,0.562865,0.0,0.437135,11,0.000667


In [90]:
def make_pop(df):
    df = df.iloc[:,1:8].multiply(df['tot_pop'], axis='index')
    return df.sum(axis=0)

In [91]:
religion2taz = religion2taz.groupby(by='new_id').apply(lambda df: make_pop(df))

In [92]:
religion2taz

,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish
new_id,,,,,,,
11,0.000542,0.000000e+00,0.000000e+00,0.046991,0.002961,0.000000e+00,0.047534
11_betshemesh,0.014474,0.000000e+00,0.000000e+00,0.067108,0.001004,0.000000e+00,0.081582
21,0.001775,0.000000e+00,0.000000e+00,0.079796,0.003972,0.000000e+00,0.081571
22,0.000266,0.000000e+00,0.000000e+00,0.022539,0.000423,0.000000e+00,0.022805
23,0.000849,0.000000e+00,0.000000e+00,0.023902,0.000428,0.000000e+00,0.024751
24,0.000871,0.000000e+00,0.000000e+00,0.039529,0.002143,0.000000e+00,0.040399
29,0.003438,0.000000e+00,0.000000e+00,0.049392,0.000756,0.000000e+00,0.052830
31,0.000050,0.000000e+00,0.000000e+00,0.021528,0.011660,0.000000e+00,0.021579
32,0.011846,0.000000e+00,0.000000e+00,0.053569,0.004420,0.000000e+00,0.065415


In [93]:
tmp = religion2taz[['Druze', 'Other', 'Muslim', 'Christian', 'Jewish']].sum(axis=1)
tmp.loc[tmp == 0] = 1
religion2taz = religion2taz.divide(tmp, axis=0)
religion2taz

,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish
new_id,,,,,,,
11,0.010743,0.000000,0.000000e+00,0.930618,0.058639,0.000000e+00,0.941361
11_betshemesh,0.175258,0.000000,0.000000e+00,0.812591,0.012151,0.000000e+00,0.987849
21,0.020747,0.000000,0.000000e+00,0.932818,0.046435,0.000000e+00,0.953565
22,0.011432,0.000000,0.000000e+00,0.970370,0.018198,0.000000e+00,0.981802
23,0.033713,0.000000,0.000000e+00,0.949281,0.017006,0.000000e+00,0.982994
24,0.020463,0.000000,0.000000e+00,0.929163,0.050374,0.000000e+00,0.949626
29,0.064165,0.000000,0.000000e+00,0.921736,0.014099,0.000000e+00,0.985901
31,0.001519,0.000000,0.000000e+00,0.647693,0.350788,0.000000e+00,0.649212
32,0.169624,0.000000,0.000000e+00,0.767082,0.063294,0.000000e+00,0.936706


In [94]:
religion2taz.reset_index(inplace=True)

In [95]:
religion2taz.columns = ['cell_id', 'Orthodox', 'Druze', 'Other', 'Sacular', 'Muslim', 'Christian', 'Jewish']
religion2taz

,cell_id,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish
0,11,0.010743,0.000000,0.000000e+00,0.930618,0.058639,0.000000e+00,0.941361
1,11_betshemesh,0.175258,0.000000,0.000000e+00,0.812591,0.012151,0.000000e+00,0.987849
2,21,0.020747,0.000000,0.000000e+00,0.932818,0.046435,0.000000e+00,0.953565
3,22,0.011432,0.000000,0.000000e+00,0.970370,0.018198,0.000000e+00,0.981802
4,23,0.033713,0.000000,0.000000e+00,0.949281,0.017006,0.000000e+00,0.982994
5,24,0.020463,0.000000,0.000000e+00,0.929163,0.050374,0.000000e+00,0.949626
6,29,0.064165,0.000000,0.000000e+00,0.921736,0.014099,0.000000e+00,0.985901
7,31,0.001519,0.000000,0.000000e+00,0.647693,0.350788,0.000000e+00,0.649212
8,32,0.169624,0.000000,0.000000e+00,0.767082,0.063294,0.000000e+00,0.936706
9,41,0.411048,0.000000,0.000000e+00,0.584662,0.004289,0.000000e+00,0.995711


In [96]:
religion2taz.to_csv('../Data/demograph/religion_dis.csv')

## Creating stay home/no_school.csv

In [129]:
#TODO

## Creating stay home/no_work.csv

In [130]:
#TODO

## Creating stay home/routine.csv

In [131]:
#TODO

# Data Loading

In [2]:
# import data:
with (open('../Data/division_choice/'+ cell_name + '/mat_macro_model_df.pickle', 'rb')) as openfile:
    OD_dict = pickle.load(openfile)

base_leisure = pd.read_csv('../Data/raw/leisure_mtx.csv',index_col=0)
base_work = pd.read_csv('../Data/raw/work_mtx.csv',index_col=0)
base_school = pd.read_csv('../Data/raw/school_mtx.csv',index_col=0)

age_dist_area = pd.read_csv('../Data/demograph/age_dist_area.csv',index_col=0)
stay_home_idx_school =  pd.read_csv('../Data/stay home/no_school.csv',index_col=0)
stay_home_idx_work =  pd.read_csv('../Data/stay home/no_work.csv',index_col=0)
stay_home_idx_routine =  pd.read_csv('../Data/stay home/routine.csv',index_col=0)
religion_dist = pd.read_csv('../Data/demograph/religion_dis.csv',index_col=0)


In [3]:
home_secularism = pd.read_excel('../Data/raw/secularism_base_home.xlsx',index_col=0)
home_haredi = pd.read_excel('../Data/raw/haredi_base_home.xlsx',index_col=0)
home_arabs = pd.read_excel('../Data/raw/arabs_base_home.xlsx',index_col=0)

In [4]:
OD_dict.keys()

dict_keys([('no_school', 0), ('no_school', 1), ('no_school', 2), ('no_work', 0), ('no_work', 1), ('no_work', 2), ('routine', 0), ('routine', 1), ('routine', 2)])

In [5]:
age_dist_area

,cell_id,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
0,11,0.003902,0.003036,0.004663,0.007137,0.010536,0.006631,0.004630,0.004642,0.005318
1,11_betshemesh,0.008565,0.007406,0.011894,0.010607,0.011555,0.009676,0.007175,0.007429,0.008280
2,21,0.007581,0.007176,0.011858,0.010028,0.011560,0.011409,0.008448,0.008761,0.008723
3,22,0.001892,0.001742,0.003223,0.003102,0.003015,0.003052,0.002524,0.002503,0.002174
4,23,0.002338,0.002074,0.003164,0.003044,0.003504,0.003166,0.002388,0.002851,0.002650
5,24,0.003750,0.003418,0.005571,0.005223,0.005631,0.005232,0.004225,0.004772,0.004720
6,29,0.005077,0.005115,0.009129,0.006794,0.006439,0.007179,0.005575,0.004375,0.003904
7,31,0.003006,0.003219,0.006285,0.004540,0.003858,0.004654,0.003392,0.002380,0.001905
8,32,0.008057,0.008017,0.013703,0.009951,0.008802,0.007986,0.005574,0.004095,0.003650
9,41,0.007217,0.006297,0.011206,0.008920,0.006004,0.004405,0.003556,0.002967,0.003079


In [6]:
base_leisure

,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
0-4,1.910947,0.785434,0.605625,0.947981,1.217311,0.539321,0.526174,0.327925,0.136674
5-9,0.855218,3.058522,1.270115,0.586938,0.994774,0.548865,0.307890,0.281533,0.096498
10-19,0.188896,0.833542,5.118978,0.980886,0.664389,0.573172,0.223943,0.129148,0.077123
20-29,0.216756,0.165483,1.147046,2.721855,1.030819,0.491379,0.301705,0.088583,0.054323
30-39,0.263324,0.231474,0.380568,0.938731,1.367406,0.614261,0.391434,0.205911,0.073415
40-49,0.098300,0.127266,0.312787,0.458589,0.688429,0.540939,0.255728,0.131058,0.066109
50-59,0.086537,0.105663,0.236436,0.604374,0.589119,0.457537,0.457947,0.236600,0.077892
60-69,0.072166,0.081541,0.124618,0.379717,0.528686,0.365015,0.373055,0.358650,0.143989
70+,0.035686,0.043422,0.112453,0.160497,0.271909,0.236009,0.190299,0.271175,0.190983


In [7]:
OD_dict['routine', 1].index

Index(['11', '11_betshemesh', '21', '22', '23', '24', '29', '31', '32', '41',
       '42', '43', '44', '51', '51_tlv', '51_bb', '61', '62', '62_arab', '71'],
      dtype='object')

In [8]:
stay_home_idx_work

,mean,std,min,max
cell_id,,,,
11,0.269553,0.060183,0.163246,0.313846
21,0.305619,0.067354,0.172872,0.354417
22,0.379769,0.051840,0.280759,0.419792
23,0.357335,0.046417,0.270225,0.393955
24,0.388517,0.053071,0.293302,0.444552
29,0.283014,0.043141,0.197223,0.317801
31,0.266843,0.074133,0.130012,0.324173
32,0.334017,0.063844,0.217401,0.382464
41,0.305952,0.066156,0.183571,0.356831


In [9]:
MI = {i: group for i, group in enumerate(itertools.product(A.values(), G.values(),  A.values()))}

In [10]:
MI

{0: ('0-4', '11', '0-4'),
 1: ('0-4', '11', '5-9'),
 2: ('0-4', '11', '10-19'),
 3: ('0-4', '11', '20-29'),
 4: ('0-4', '11', '30-39'),
 5: ('0-4', '11', '40-49'),
 6: ('0-4', '11', '50-59'),
 7: ('0-4', '11', '60-69'),
 8: ('0-4', '11', '70+'),
 9: ('0-4', '11_betshemesh', '0-4'),
 10: ('0-4', '11_betshemesh', '5-9'),
 11: ('0-4', '11_betshemesh', '10-19'),
 12: ('0-4', '11_betshemesh', '20-29'),
 13: ('0-4', '11_betshemesh', '30-39'),
 14: ('0-4', '11_betshemesh', '40-49'),
 15: ('0-4', '11_betshemesh', '50-59'),
 16: ('0-4', '11_betshemesh', '60-69'),
 17: ('0-4', '11_betshemesh', '70+'),
 18: ('0-4', '21', '0-4'),
 19: ('0-4', '21', '5-9'),
 20: ('0-4', '21', '10-19'),
 21: ('0-4', '21', '20-29'),
 22: ('0-4', '21', '30-39'),
 23: ('0-4', '21', '40-49'),
 24: ('0-4', '21', '50-59'),
 25: ('0-4', '21', '60-69'),
 26: ('0-4', '21', '70+'),
 27: ('0-4', '22', '0-4'),
 28: ('0-4', '22', '5-9'),
 29: ('0-4', '22', '10-19'),
 30: ('0-4', '22', '20-29'),
 31: ('0-4', '22', '30-39'),
 32: 

# Processing stay home index

In [11]:
span = stay_home_idx_routine['max'] - stay_home_idx_work['min']
span

cell_id
11               0.386542
21               0.379869
22               0.323442
23               0.322763
24               0.317497
29               0.408615
31               0.443478
32               0.398331
41               0.433597
42               0.493654
43               0.493283
44               0.526529
51               0.497377
61               0.442709
62               0.384018
71               0.316797
11_betshemesh    0.474715
51_bb            0.343853
51_tlv           0.440693
62_arab          0.236984
dtype: float64

In [12]:
stay_home_idx_work['percent_idx'] = 1 - ((stay_home_idx_routine['max'] - stay_home_idx_work['mean'])/span)
stay_home_idx_school['percent_idx'] = 1 - ((stay_home_idx_routine['max'] - stay_home_idx_school['mean'])/span)

In [13]:
stay_home_idx_work

,mean,std,min,max,percent_idx
cell_id,,,,,
11,0.269553,0.060183,0.163246,0.313846,0.275023
21,0.305619,0.067354,0.172872,0.354417,0.349454
22,0.379769,0.051840,0.280759,0.419792,0.306116
23,0.357335,0.046417,0.270225,0.393955,0.269888
24,0.388517,0.053071,0.293302,0.444552,0.299893
29,0.283014,0.043141,0.197223,0.317801,0.209954
31,0.266843,0.074133,0.130012,0.324173,0.308539
32,0.334017,0.063844,0.217401,0.382464,0.292761
41,0.305952,0.066156,0.183571,0.356831,0.282247


In [14]:
stay_home_idx_school

,min,max,mean,std,percent_idx
cell_id,,,,,
11,0.279484,0.413518,0.375147,0.060502,0.548196
21,0.284609,0.438834,0.392464,0.068269,0.578073
22,0.389827,0.502554,0.469736,0.050544,0.584272
23,0.393587,0.477372,0.452540,0.037300,0.564856
24,0.427730,0.505448,0.482527,0.034667,0.595990
29,0.358212,0.447196,0.420178,0.039236,0.545636
31,0.292136,0.436327,0.394418,0.064695,0.596210
32,0.369315,0.484902,0.449557,0.050824,0.582821
41,0.339401,0.467383,0.430178,0.057417,0.568748


# Leisure Matrix

## routine

In [19]:
full_leisure_routine = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()), names=['age','area','age']), 
                                    columns=OD_dict['routine', 2].index)

In [20]:
# fill the matrix:
for index in list(full_leisure_routine.index):
    tmp = age_dist_area[index[2]]
    tmp.index=list(G.values())
    full_leisure_routine.loc[index] = (OD_dict['routine', 2].loc[index[1]] * base_leisure.loc[index[0]][index[2]]) / tmp

In [21]:
full_leisure_routine

11 11_betshemesh           21          22  \
age area age                                                         
0-4 11   0-4        489.289     0.0744188    0.0118863   0.0528482   
         5-9        258.471     0.0353763   0.00516162   0.0235876   
         10-19      129.747     0.0169846   0.00240841  0.00983377   
         20-29      132.685     0.0298112   0.00445808   0.0159939   
         30-39       115.42       0.03514   0.00496596   0.0211254   
...                     ...           ...          ...         ...   
70+ 71   30-39  0.000829609    0.00285686   0.00110269  0.00553498   
         40-49   0.00114409     0.0029611  0.000969729  0.00474642   
         50-59   0.00132122    0.00321979   0.00105603  0.00462792   
         60-69   0.00187785    0.00443169   0.00145101  0.00665067   
         70+     0.00115445    0.00280023   0.00102632  0.00539108   

                        23           24          29           31           32  \
age area age                                                                    
0-4 11   0-4     0.0126726   0.00503233    0.023545   0.00664766   0.00490477   
         5-9    0.00587211   0.00226924  0.00960537    0.0025516    0.0020261   
         10-19  0.00296793   0.00107372  0.00414972   0.00100765  0.000914039   
         20-29  0.00482842   0.00179237   0.0087279   0.00218365   0.00197018   
         30-39  0.00538673   0.00213498   0.0118257   0.00329969   0.00285999   
...                    ...          ...         ...          ...          ...   
70+ 71   30-39  0.00317217  0.000548936  0.00370886  0.000676286  0.000891248   
         40-49    0.003047   0.00051279  0.00288722  0.000486615   0.00085263   
         50-59  0.00325803  0.000512032  0.00299808  0.000538342   0.00098498   
         60-69  0.00388886  0.000646065  0.00544407   0.00109326   0.00191051   
         70+    0.00294568  0.000460009   0.0042966  0.000962061   0.00150983   

                         41          42          43          44          51  \
age area age                                                                  
0-4 11   0-4      0.0048176  0.00797166   0.0114426   0.0521304   0.0187842   
         5-9     0.00226945  0.00376286  0.00509846   0.0210364  0.00803874   
         10-19  0.000983228   0.0018229  0.00285626  0.00919976  0.00489258   
         20-29   0.00193343  0.00300277   0.0073901   0.0199554   0.0098276   
         30-39   0.00368875  0.00432327   0.0107324   0.0311703   0.0125959   
...                     ...         ...         ...         ...         ...   
70+ 71   30-39   0.00125578  0.00126702  0.00258242  0.00552172  0.00237093   
         40-49   0.00148578  0.00138988  0.00316181  0.00537552  0.00289643   
         50-59   0.00148386  0.00127438  0.00432709  0.00602232  0.00341304   
         60-69   0.00253436  0.00186809   0.0091684   0.0136293  0.00588429   
         70+      0.0017198  0.00125408  0.00908991   0.0167354  0.00839786   

                     51_tlv       51_bb          61           62     62_arab  \
age area age                                                                   
0-4 11   0-4       0.013218    0.191965   0.0170772   0.00352959   0.0242156   
         5-9     0.00558724   0.0893819   0.0068379   0.00144877   0.0102911   
         10-19   0.00222625   0.0444639  0.00254567   0.00056918  0.00454866   
         20-29   0.00388719   0.0554832   0.0046969   0.00104975  0.00844073   
         30-39   0.00558261   0.0826496  0.00732348   0.00173848   0.0107838   
...                     ...         ...         ...          ...         ...   
70+ 71   30-39  0.000381191  0.00947701  0.00303873  0.000486689  0.00326304   
         40-49   0.00034026   0.0104224  0.00259671  0.000435267  0.00300435   
         50-59  0.000326864  0.00919692  0.00261936  0.000438093  0.00309773   
         60-69  0.000519022   0.0132659  0.00527941  0.000861171  0.00506154   
         70+    0.000388637  0.00826075   0.0044401  0.000775535  0.0039

In [22]:
# save matrix
try:
    os.mkdir('../Data/base_contact_mtx')
except:
    pass
full_leisure_routine.unstack().reorder_levels(['area','age']).sort_index().to_csv(
    '../Data/base_contact_mtx/full_leisure_mtx_routine.csv')

In [23]:
full_leisure_routine.unstack().reorder_levels(['area','age']).sort_index()

11                                                     \
age                 0-4        10-19        20-29        30-39       40-49   
area age                                                                     
11   0-4        489.289      129.747      132.685       115.42     81.2478   
     10-19       48.366      1096.67      137.291      62.9947     86.3474   
     20-29      55.4992      245.739      380.968       97.738     74.0254   
     30-39       67.423      81.5313      131.391      129.652     92.5374   
     40-49      25.1692      67.0102       64.187       65.274     81.4915   
...                 ...          ...          ...          ...         ...   
71   40-49  0.000809911    0.0021563   0.00206545   0.00210043  0.00262229   
     5-9     0.00704631   0.00875596   0.00264353    0.0030351  0.00266071   
     50-59  0.000712998   0.00162995   0.00272206   0.00179743  0.00221798   
     60-69  0.000594589  0.000859095   0.00171022   0.00161305  0.00176947   
     70+    0.000294022  0.000775234  0.000722867  0.000829609  0.00114409   

                                                              11_betshemesh  \
age                 5-9       50-59        60-69          70+           0-4   
area age                                                                      
11   0-4        258.471     113.527      70.5694      25.6743     0.0744188   
     10-19      274.303     48.3177      27.7926      14.4877    0.00735626   
     20-29      54.4575     65.0956      19.0629      10.2047     0.0084412   
     30-39      76.1736     84.4555      44.3119      13.7911     0.0102547   
     40-49      41.8809     55.1758      28.2036      12.4185    0.00382813   
...                 ...         ...          ...          ...           ...   
71   40-49   0.00134767  0.00177548  0.000907555  0.000399612    0.00139332   
     5-9      0.0323879  0.00213764   0.00194957  0.000583309     0.0121221   
     50-59   0.00111891  0.00317946   0.00163842   0.00047084     0.0012266   
     60-69  0.000863473  0.00259007   0.00248359  0.000870385     0.0010229   
     70+    0.000459807  0.00132122   0.00187785   0.00115445   0.000505818   

            ...      62_arab           71                           \
age         ...          70+          0-4        10-19       20-29   
area age    ...                                                      
11   0-4    ...   0.00208312   0.00485724   0.00102523  0.00180615   
     10-19  ...   0.00117548  0.000480136   0.00866566  0.00186884   
     20-29  ...  0.000827972  0.000550949   0.00194178  0.00518584   
     30-39  ...   0.00111896  0.000669318  0.000644243  0.00178852   
     40-49  ...    0.0010076  0.000249858    0.0005295  0.00087373   
...         ...          ...          ...          ...         ...   
71   40-49  ...   0.00136494      7.72098      16.3623     26.9995   
     5-9    ...   0.00199239      67.1733      66.4416     34.5562   
     50-59  ...   0.00160823       6.7971      12.3683     35.5827   
     60-69  ...   0.00297294      5.66829      6.51895     22.3559   
     70+    ...   0.00394322      2.80295       5.8826      9.4493   

                                                                           \
age              30-39       40-49          5-9        50-59        60-69   
area age                                                                    
11   0-4    0.00274656  0.00146372   0.00224825   0.00166988   0.00122489   
     10-19  0.00149903  0.00155559   0.00238596  0.000710712  0.000482406   
     20-29  0.00232579  0.00133361  0.000473686    0.0009575  0.000330881   
     30-39  0.00308521  0.00166711  0.000662579   0.00124227  0.000769135   
     40-49  0.00155327  0.00146811  0.000364291  0.000811589  0.000489539   
...                ...         ...          ...          ...          ...   
71   40-49     47.9983     45.3667      11.2571      25.0793      15.1275   
     5-9       69.3571     46.0315      270.537      30.1948      32.4

## No school

In [24]:
full_leisure_no_school = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), 
                                     columns=OD_dict['no_school', 2].index)

In [26]:
# fill the matrix:
for index in list(full_leisure_no_school.index):
    tmp = age_dist_area[index[2]]
    tmp.index=list(G.values())
    full_leisure_no_school.loc[index] = (OD_dict['no_school', 2].loc[index[1]] \
                                         * stay_home_idx_school.loc[index[1]]['percent_idx'] \
                                         * base_leisure.loc[index[0]][index[2]]) / tmp

In [27]:
full_leisure_no_school.unstack().reorder_levels(['area','age']).sort_index()

11                                                      \
age                 0-4        10-19        20-29        30-39        40-49   
area age                                                                      
11   0-4        268.258       71.135      72.7462      63.2805      44.5449   
     10-19      26.5172      601.261      75.2712      34.5375      47.3409   
     20-29       30.428      134.729       208.87      53.5859      40.5852   
     30-39      36.9653      44.7004      72.0363       71.083      50.7346   
     40-49      13.7993       36.739      35.1912      35.7872      44.6785   
...                 ...          ...          ...          ...          ...   
71   40-49  0.000440024   0.00117152   0.00112216   0.00114116   0.00142469   
     5-9     0.00382826   0.00475711   0.00143623   0.00164897   0.00144556   
     50-59  0.000387372  0.000885553   0.00147889  0.000976545   0.00120503   
     60-69   0.00032304  0.000466746  0.000929161  0.000876369  0.000961353   
     70+    0.000159742  0.000421185  0.000392733  0.000450726  0.000621585   

                                                               11_betshemesh  \
age                 5-9        50-59        60-69          70+           0-4   
area age                                                                       
11   0-4         141.71      62.2423      38.6904      14.0762     0.0375508   
     10-19      150.389      26.4907      15.2376      7.94301    0.00371188   
     20-29      29.8569      35.6893      10.4515      5.59483    0.00425933   
     30-39       41.763      46.3036      24.2945       7.5611    0.00517442   
     40-49      22.9616      30.2507      15.4629      6.80858    0.00193163   
...                 ...          ...          ...          ...           ...   
71   40-49  0.000732189   0.00096462  0.000493074  0.000217109   0.000679448   
     5-9      0.0175963   0.00116138    0.0010592  0.000316912    0.00591127   
     50-59  0.000607903    0.0017274  0.000890151  0.000255807   0.000598146   
     60-69  0.000469125   0.00140718   0.00134933   0.00047288   0.000498811   
     70+    0.000249813  0.000717818   0.00102023  0.000627214    0.00024666   

            ...      62_arab           71                            \
age         ...          70+          0-4        10-19        20-29   
area age    ...                                                       
11   0-4    ...   0.00107522   0.00263186  0.000555514  0.000978649   
     10-19  ...  0.000606735  0.000260158   0.00469542   0.00101262   
     20-29  ...  0.000427366  0.000298528   0.00105214   0.00280991   
     30-39  ...  0.000577562  0.000362665  0.000349078  0.000969099   
     40-49  ...   0.00052008  0.000135384  0.000286906  0.000473424   
...         ...          ...          ...          ...          ...   
71   40-49  ...  0.000774022      4.32393       9.1633      15.1204   
     5-9    ...   0.00112983      37.6187      37.2089      19.3523   
     50-59  ...  0.000911986      3.80654      6.92656      19.9272   
     60-69  ...   0.00168588      3.17438      3.65077      12.5199   
     70+    ...    0.0022361      1.56972       3.2944      5.29183   

                                                                             \
age               30-39        40-49          5-9        50-59        60-69   
area age                                                                      
11   0-4      0.0014882  0.000793106    0.0012182  0.000904814    0.0006637   
     10-19  0.000812238  0.000842886   0.00129281  0.000385094  0.000261388   
     20-29   0.00126021  0.000722604  0.000256663  0.000518814  0.000179286   
     30-39    0.0016717   0.00090331  0.000359013  0.000673114   0.00041675   
     40-49  0.000841627  0.000795485  0.000197388  0.000439754  0.000265253   
...                 ...          ...          ...          ...          ...   
71   40-49      26.8802      25.4065      6.30426       14.045      8.47174   
     5-9

In [28]:
# save matrix
try:
    os.mkdir('../Data/base_contact_mtx')
except:
    pass
full_leisure_no_school.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_leisure_mtx_no_school.csv')

## No work

In [29]:
full_leisure_no_work = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), 
                                     columns=OD_dict['no_work', 2].index)

In [30]:
# fill the matrix:
for index in list(full_leisure_no_work.index):
    tmp = age_dist_area[index[2]]
    tmp.index=list(G.values())
    full_leisure_no_work.loc[index] = (OD_dict['no_work', 2].loc[index[1]] * \
                                       stay_home_idx_work.loc[index[1]]['percent_idx'] * \
                                       base_leisure.loc[index[0]][index[2]]) / tmp

In [31]:
# save matrix
full_leisure_no_work.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_leisure_mtx_no_work.csv')

In [32]:
full_leisure_no_work.unstack().reorder_levels(['area','age']).sort_index()

11                                                      \
age                 0-4        10-19        20-29        30-39        40-49   
area age                                                                      
11   0-4        134.611      35.6952      36.5037      31.7539      22.3525   
     10-19      13.3062       301.71      37.7708      17.3308      23.7555   
     20-29      15.2687      67.6064       104.81      26.8892      20.3655   
     30-39       18.549      22.4305      36.1475      35.6691      25.4584   
     40-49      6.92441      18.4355      17.6588      17.9578      22.4195   
...                 ...          ...          ...          ...          ...   
71   40-49  0.000194189  0.000517008  0.000495225  0.000503612  0.000628736   
     5-9     0.00168947   0.00209938  0.000633829  0.000727715  0.000637949   
     50-59  0.000170953  0.000390808  0.000652658  0.000430964  0.000531797   
     60-69  0.000142562  0.000205982  0.000410052  0.000386755  0.000424259   
     70+    7.04965e-05  0.000185875  0.000173319  0.000198912  0.000274315   

                                                               11_betshemesh  \
age                 5-9        50-59        60-69          70+           0-4   
area age                                                                       
11   0-4        71.1093      31.2329      19.4147      7.06338     0.0148253   
     10-19      75.4647      13.2929      7.64617      3.98577    0.00146547   
     20-29       14.982      17.9087      5.24449      2.80746     0.0016816   
     30-39      20.9565      23.2349      12.1909      3.79413    0.00204289   
     40-49       11.522      15.1797      7.75923      3.41652   0.000762616   
...                 ...          ...          ...          ...           ...   
71   40-49  0.000323126  0.000425701  0.000217601  9.58133e-05   0.000328231   
     5-9     0.00776552  0.000512533  0.000467441  0.000139858    0.00285564   
     50-59  0.000268277  0.000762327  0.000392837  0.000112891   0.000288955   
     60-69  0.000207032  0.000621011  0.000595481  0.000208689   0.000240968   
     70+    0.000110246  0.000316784  0.000450244  0.000276799   0.000119158   

            ...      62_arab           71                            \
age         ...          70+          0-4        10-19        20-29   
area age    ...                                                       
11   0-4    ...   0.00050269   0.00125788  0.000265504  0.000467739   
     10-19  ...  0.000283661  0.000124341   0.00224414  0.000483974   
     20-29  ...  0.000199802  0.000142679  0.000502862   0.00134298   
     30-39  ...  0.000270022  0.000173333   0.00016684  0.000463174   
     40-49  ...  0.000243148  6.47057e-05  0.000137125   0.00022627   
...         ...          ...          ...          ...          ...   
71   40-49  ...  0.000363124       1.8868      3.99852      6.59797   
     5-9    ...  0.000530048      16.4154      16.2366      8.44462   
     50-59  ...  0.000427848      1.66103       3.0225      8.69548   
     60-69  ...  0.000790911      1.38518      1.59306       5.4632   
     70+    ...   0.00104904     0.684966      1.43755      2.30916   

                                                                             \
age               30-39        40-49          5-9        50-59        60-69   
area age                                                                      
11   0-4    0.000711276  0.000379059   0.00058223   0.00043245  0.000317211   
     10-19  0.000388204  0.000402852  0.000617892  0.000184053  0.000124929   
     20-29  0.000602309  0.000345364   0.00012267  0.000247964  8.56883e-05   
     30-39  0.000798977  0.000431731  0.000171588   0.00032171  0.000199183   
     40-49   0.00040225  0.000380196  9.43405e-05  0.000210177  0.000126776   
...                 ...          ...          ...          ...          ...   
71   40-49      11.7295      11.0864      2.75095      6.12872      3.69675   
     5-9

# Work Matrix

In [33]:
# creating school- work matrix;
base_work_school = base_work.copy()
base_work_school.loc['0-4'] = base_school.loc['0-4']
base_work_school.loc['5-9'] = base_school.loc['5-9']
base_work_school['0-4'] = base_school['0-4']
base_work_school['5-9'] = base_school['5-9']

In [34]:
base_work_school

,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
0-4,2.901178,0.353363,0.117947,0.027197,0.070175,0.030752,0.016916,0.002066,0.000000
5-9,0.442161,2.801755,0.163184,0.019576,0.036961,0.030238,0.015680,0.003323,0.000347
10-19,0.010633,0.311794,0.364628,0.425551,0.307760,0.299323,0.114176,0.006199,0.000004
20-29,0.007126,0.011894,0.395992,1.726451,1.530541,1.131590,0.606490,0.044159,0.000015
30-39,0.019030,0.054146,0.315675,1.294422,2.078599,1.751081,0.800303,0.041232,0.000011
40-49,0.034396,0.036878,0.293957,1.110030,1.765236,1.937622,0.943991,0.044619,0.000020
50-59,0.038944,0.099026,0.269529,0.761644,1.279458,1.585571,1.019917,0.049350,0.000018
60-69,0.019908,0.024630,0.022494,0.094078,0.146233,0.167043,0.121437,0.006846,0.000030
70+,0.000000,0.014121,0.000009,0.000075,0.000113,0.000048,0.000029,0.000021,0.000014


In [35]:
OD_kids = OD_dict['routine', 1].copy()

for col in OD_kids.columns:
    OD_kids[col].values[:] = 0
OD_kids.values[[np.arange(OD_kids.shape[0])]*2] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """


In [36]:
OD_kids.head()

,11,11_betshemesh,21,22,23,24,29,31,32,41,42,43,44,51,51_tlv,51_bb,61,62,62_arab,71
11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11_betshemesh,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Routine

In [37]:
full_work_routine = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), columns=OD_dict['routine', 1].index)

In [38]:
# fill the matrix:
for index in list(full_work_routine.index):
    if index[0] in['0-4', '5-9']:
        full_work_routine.loc[index] = (OD_kids.loc[index[1]] * base_work_school.loc[index[0]][index[2]]) / age_dist_area[index[2]]
    else:
        full_work_routine.loc[index] = (OD_dict['routine', 1].loc[index[1]] * base_work_school.loc[index[0]][index[2]]) / age_dist_area[index[2]]

In [40]:
# save matrix
full_work_routine.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_work_mtx_routine.csv')

In [39]:
full_work_routine.unstack().reorder_levels(['area','age']).sort_index()

11                                                      \
age                 0-4        10-19        20-29        30-39        40-49   
area age                                                                      
11   0-4        743.591      25.2942      3.81061      6.66049      4.63741   
     10-19      2.72188      78.0955      59.5469      29.1727      45.0804   
     20-29      1.82401      84.8131       241.58      145.081      170.427   
     30-39       4.8713      67.6108      181.127      197.032      263.727   
     40-49      8.80468      62.9594      155.325      167.328      291.821   
...                 ...          ...          ...          ...          ...   
71   40-49  0.000320234   0.00228989   0.00564933   0.00608587    0.0106138   
     5-9              0            0            0            0            0   
     50-59  0.000362574    0.0020996   0.00387627   0.00441109   0.00868537   
     60-69  0.000185345  0.000175227  0.000478794  0.000504156  0.000915022   
     70+              0  7.38481e-08  3.82719e-07  3.89143e-07  2.60741e-07   

                                                               11_betshemesh  \
age                 5-9        50-59        60-69          70+           0-4   
area age                                                                       
11   0-4        116.404       3.6536     0.445018            0             0   
     10-19      102.578      24.6279      1.33371  0.000741811   0.000475806   
     20-29      3.91291      130.821      9.50051   0.00283578   0.000318852   
     30-39      17.8137      172.627      8.87074   0.00212214   0.000851542   
     40-49      12.1326       203.62      9.59952   0.00381235    0.00153913   
...                 ...          ...          ...          ...           ...   
71   40-49  0.000441275   0.00740586  0.000349144  1.38659e-07   0.000436188   
     5-9              0            0            0            0             0   
     50-59   0.00118492   0.00800153  0.000386163  1.20899e-07   0.000493858   
     60-69  0.000294722  0.000952706  5.35728e-05  2.02182e-07   0.000252456   
     70+    0.000168964  2.28297e-07  1.65106e-07  9.69927e-08             0   

            ...      62_arab           71                           \
age         ...          70+          0-4        10-19       20-29   
area age    ...                                                      
11   0-4    ...            0            0            0           0   
     10-19  ...  9.41752e-08  3.38977e-05  0.000774156  0.00101687   
     20-29  ...  3.60011e-07  2.27159e-05  0.000840747  0.00412542   
     30-39  ...  2.69412e-07  6.06661e-05  0.000670222  0.00309307   
     40-49  ...  4.83989e-07  0.000109651  0.000624112  0.00265246   
...         ...          ...          ...          ...         ...   
71   40-49  ...  4.98892e-07      2.70129      15.3751      65.344   
     5-9    ...            0      34.7574       8.5432     1.15348   
     50-59  ...  4.34994e-07      3.05844      14.0974     44.8356   
     60-69  ...  7.27448e-07      1.56345      1.17653     5.53806   
     70+    ...  3.48978e-07            0  0.000495842  0.00442679   

                                                                            \
age               30-39       40-49          5-9        50-59        60-69   
area age                                                                     
11   0-4              0           0            0            0            0   
     10-19  0.000870883  0.00101885   0.00111934  0.000454456  2.90415e-05   
     20-29   0.00433105  0.00385177  4.26982e-05   0.00241402  0.000206874   
     30-39   0.00588191  0.00596043  0.000194385   0.00318546  0.000193161   
     40-49   0.00499518  0.00659539  0.000132393   0.00375738   0.00020903   
...                 ...         ...          ...          ...          ...   
71   40-49      123.057     162.479      3.26153       92.564       5.1495   
     5-9          2.579     2.53

## No School

In [41]:
full_work_no_school = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), columns=OD_dict['no_school', 1].index)

In [42]:
# fill the matrix:
for index in list(full_work_no_school.index):
    if index[0] in['0-4', '5-9']:
        full_work_no_school.loc[index] = (OD_kids.loc[index[1]] * base_work.loc[index[0]][index[2]]) / age_dist_area[index[2]]
    else:
        full_work_no_school.loc[index] = (OD_dict['no_school', 1].loc[index[1]] * stay_home_idx_school.loc[index[1]]['percent_idx'] * base_work.loc[index[0]][index[2]]) / age_dist_area[index[2]]

In [44]:
# save matrix
full_work_no_school.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_work_mtx_no_school.csv')

In [43]:
full_work_no_school.unstack().reorder_levels(['area','age']).sort_index()

11                                                      \
age                 0-4        10-19        20-29        30-39        40-49   
area age                                                                      
11   0-4              0            0            0            0            0   
     10-19            0      42.8118      32.6435      15.9924      24.7129   
     20-29            0      46.4943      132.434      79.5329      93.4274   
     30-39            0      37.0641      99.2934      108.012      144.574   
     40-49            0      34.5142      85.1489      91.7286      159.976   
...                 ...          ...          ...          ...          ...   
71   40-49            0   0.00141835   0.00349917   0.00376956   0.00657415   
     5-9              0            0            0            0            0   
     50-59            0   0.00130048   0.00240094   0.00273221   0.00537968   
     60-69  2.08753e-08  0.000108535  0.000296563  0.000312272   0.00056676   
     70+              0  4.57412e-08  2.37054e-07  2.41034e-07  1.61502e-07   

                                                               11_betshemesh  \
age                 5-9        50-59        60-69          70+           0-4   
area age                                                                       
11   0-4              0            0            0   0.00227531             0   
     10-19            0      13.5009     0.731134  0.000406659             0   
     20-29            0      71.7157      5.20815   0.00155457             0   
     30-39            0      94.6334      4.86291   0.00116335             0   
     40-49            0      111.624      5.26243   0.00208992             0   
...                 ...          ...          ...          ...           ...   
71   40-49            0   0.00458716  0.000216258  8.58845e-08             0   
     5-9              0            0            0            0             0   
     50-59            0   0.00495611  0.000239188  7.48845e-08             0   
     60-69  1.18585e-08  0.000590102  3.31828e-05  1.25231e-07   3.15184e-08   
     70+              0  1.41406e-07  1.02266e-07  6.00769e-08             0   

            ...      62_arab           71                            \
age         ...          70+          0-4        10-19        20-29   
area age    ...                                                       
11   0-4    ...            0            0            0            0   
     10-19  ...  5.19046e-08            0  0.000400644  0.000526255   
     20-29  ...   1.9842e-07            0  0.000435106     0.002135   
     30-39  ...  1.48487e-07            0  0.000346855   0.00160074   
     40-49  ...   2.6675e-07            0  0.000322992   0.00137271   
...         ...          ...          ...          ...          ...   
71   40-49  ...  2.89178e-07            0      8.60902      36.5881   
     5-9    ...            0            0            0            0   
     50-59  ...  2.52141e-07            0      7.89359      25.1048   
     60-69  ...  4.21659e-07  0.000159185     0.658777      3.10093   
     70+    ...  2.02282e-07            0  0.000277637    0.0024787   

                                                                             \
age               30-39        40-49          5-9        50-59        60-69   
area age                                                                      
11   0-4              0            0            0            0            0   
     10-19  0.000450702  0.000527279            0  0.000235191  1.50296e-05   
     20-29   0.00224142   0.00199338            0   0.00124931  0.000107062   
     30-39   0.00304403   0.00308466            0   0.00164855  9.99651e-05   
     40-49   0.00258512   0.00341326            0   0.00194453  0.000108178   
...                 ...          ...          ...          ...          ...   
71   40-49      68.9036      90.9769            0      51.8294      2.88336   
     5-9

## No Work

In [45]:
full_work_no_school_work = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), columns=OD_dict['no_work', 1].index)

In [46]:
for index in list(full_work_no_school_work.index):
    if index[0] in['0-4', '5-9']:
        full_work_no_school_work.loc[index] = (OD_kids.loc[index[1]] * base_work.loc[index[0]][index[2]]) / age_dist_area[index[2]]
    else:
        full_work_no_school_work.loc[index] = (OD_dict['no_work', 1].loc[index[1]] * stay_home_idx_work.loc[index[1]]['percent_idx'] * base_work.loc[index[0]][index[2]]) / age_dist_area[index[2]]

In [47]:
full_work_no_school_work.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_work_mtx_no_work.csv')

In [48]:
full_work_no_school_work.unstack().reorder_levels(['area','age']).sort_index()

11                                                      \
age                 0-4        10-19        20-29        30-39        40-49   
area age                                                                      
11   0-4              0            0            0            0            0   
     10-19            0      21.4842      16.3814      8.02545      12.4017   
     20-29            0      23.3322       66.459      39.9119      46.8845   
     30-39            0      18.5998      49.8282      54.2036      72.5515   
     40-49            0      17.3202      42.7301       46.032      80.2803   
...                 ...          ...          ...          ...          ...   
71   40-49            0  0.000557631   0.00137572   0.00148202   0.00258466   
     5-9              0            0            0            0            0   
     50-59            0  0.000511291  0.000943944   0.00107418   0.00211505   
     60-69  8.20722e-09  4.26709e-05  0.000116595  0.000122771  0.000222825   
     70+              0  1.79834e-08  9.31992e-08  9.47637e-08  6.34953e-08   

                                                               11_betshemesh  \
age                 5-9        50-59        60-69          70+           0-4   
area age                                                                       
11   0-4              0            0            0   0.00227531             0   
     10-19            0      6.77515     0.366904  0.000204073             0   
     20-29            0       35.989       2.6136  0.000780127             0   
     30-39            0      47.4897      2.44035  0.000583804             0   
     40-49            0      56.0161      2.64084   0.00104878             0   
...                 ...          ...          ...          ...           ...   
71   40-49            0   0.00180347  8.50229e-05  3.37659e-08             0   
     5-9              0            0            0            0             0   
     50-59            0   0.00194852  9.40378e-05  2.94412e-08             0   
     60-69  4.66222e-09  0.000232002   1.3046e-05  4.92351e-08    1.1462e-08   
     70+              0  5.55947e-08  4.02063e-08  2.36195e-08             0   

            ...      62_arab           71                            \
age         ...          70+          0-4        10-19        20-29   
area age    ...                                                       
11   0-4    ...            0            0            0            0   
     10-19  ...  2.63652e-08            0  0.000179171  0.000235346   
     20-29  ...  1.00788e-07            0  0.000194583  0.000954792   
     30-39  ...  7.54245e-08            0  0.000155117  0.000715864   
     40-49  ...  1.35497e-07            0  0.000144445  0.000613888   
...         ...          ...          ...          ...          ...   
71   40-49  ...  1.26814e-07            0        3.757      15.9672   
     5-9    ...            0            0            0            0   
     50-59  ...  1.10572e-07            0      3.44479      10.9558   
     60-69  ...  1.84911e-07  6.94687e-05     0.287493      1.35326   
     70+    ...  8.87075e-08            0  0.000121162   0.00108171   

                                                                             \
age               30-39        40-49          5-9        50-59        60-69   
area age                                                                      
11   0-4              0            0            0            0            0   
     10-19  0.000201558  0.000235804            0   0.00010518  6.72139e-06   
     20-29   0.00100238  0.000891457            0  0.000558704  4.78791e-05   
     30-39   0.00136131   0.00137949            0  0.000737245  4.47053e-05   
     40-49   0.00115609   0.00152644            0  0.000869612  4.83781e-05   
...                 ...          ...          ...          ...          ...   
71   40-49      30.0698      39.7026            0      22.6185      1.25831   
     5-9

# Home Matrix

In [53]:
religion_dist.set_index('cell_id', inplace=True)
religion_dist

,Orthodox,Druze,Other,Sacular,Muslim,Christian,Jewish
cell_id,,,,,,,
11,0.010743,0.000000,0.000000e+00,0.930618,0.058639,0.000000e+00,0.941361
11_betshemesh,0.175258,0.000000,0.000000e+00,0.812591,0.012151,0.000000e+00,0.987849
21,0.020747,0.000000,0.000000e+00,0.932818,0.046435,0.000000e+00,0.953565
22,0.011432,0.000000,0.000000e+00,0.970370,0.018198,0.000000e+00,0.981802
23,0.033713,0.000000,0.000000e+00,0.949281,0.017006,0.000000e+00,0.982994
24,0.020463,0.000000,0.000000e+00,0.929163,0.050374,0.000000e+00,0.949626
29,0.064165,0.000000,0.000000e+00,0.921736,0.014099,0.000000e+00,0.985901
31,0.001519,0.000000,0.000000e+00,0.647693,0.350788,0.000000e+00,0.649212
32,0.169624,0.000000,0.000000e+00,0.767082,0.063294,0.000000e+00,0.936706


In [54]:
full_home_mtx = pd.DataFrame(index = pd.MultiIndex.from_tuples(list(MI.values()),names=['age','area','age']), columns=OD_dict['no_work', 1].index)

In [58]:
# fill the matrix:
# ask dan if there is a need to normalize the mtx
for index in list(full_work_routine.index):
    religion_area = religion_dist.loc[index[1]].copy()
    cell_val = religion_area['Orthodox'] * home_haredi.loc[index[0]][index[2]] + \
                religion_area['Sacular'] * home_secularism.loc[index[0]][index[2]] + \
                religion_area['Christian'] * home_secularism.loc[index[0]][index[2]] + \
                religion_area['Other'] * home_secularism.loc[index[0]][index[2]] + \
                religion_area['Druze'] * home_arabs.loc[index[0]][index[2]] + \
                religion_area['Muslim'] * home_arabs.loc[index[0]][index[2]]
    full_home_mtx.loc[index] = (OD_kids.loc[index[1]] * cell_val) / age_dist_area[index[2]]


In [59]:
full_home_mtx.unstack().reorder_levels(['area','age']).sort_index()

11                                                        \
age             0-4    10-19    20-29    30-39    40-49      5-9    50-59   
area age                                                                    
11   0-4    48.3854  65.1842  43.3992  27.5622  39.2282  58.8407  1.51161   
     10-19  48.3854  65.1842  43.3992  27.5622  39.2282  58.8407  1.51161   
     20-29  81.4977  110.003  53.7704  34.3779  49.4376  99.2539  34.7085   
     30-39  81.2043  109.603  66.6637  26.5398  38.0851  98.8942  22.2774   
     40-49  79.8753  108.199   66.329  26.3941  38.0851  97.5457  22.4513   
...             ...      ...      ...      ...      ...      ...      ...   
71   40-49        0        0        0        0        0        0        0   
     5-9          0        0        0        0        0        0        0   
     50-59        0        0        0        0        0        0        0   
     60-69        0        0        0        0        0        0        0   
     70+          0        0        0        0        0        0        0   

                             11_betshemesh  ... 62_arab        71            \
age           60-69      70+           0-4  ...     70+       0-4     10-19   
area age                                    ...                               
11   0-4    1.35558  1.15924             0  ...       0         0         0   
     10-19  1.35558  1.15924             0  ...       0         0         0   
     20-29  31.2067    26.74             0  ...       0         0         0   
     30-39  20.0285  17.1609             0  ...       0         0         0   
     40-49  20.3411  17.3669             0  ...       0         0         0   
...             ...      ...           ...  ...     ...       ...       ...   
71   40-49        0        0             0  ...       0   26.3163   28.2298   
     5-9          0        0             0  ...       0   16.5688   17.5909   
     50-59        0        0             0  ...       0  0.376006  0.403248   
     60-69        0        0             0  ...       0  0.376006  0.403248   
     70+          0        0             0  ...       0  0.376006  0.403248   

                                                                               
age           20-29    30-39    40-49       5-9     50-59     60-69       70+  
area age                                                                       
11   0-4          0        0        0         0         0         0         0  
     10-19        0        0        0         0         0         0         0  
     20-29        0        0        0         0         0         0         0  
     30-39        0        0        0         0         0         0         0  
     40-49        0        0        0         0         0         0         0  
...             ...      ...      ...       ...       ...       ...       ...  
71   40-49    32.24  19.5109  21.2214   28.0489    9.9832   10.7705   11.8231  
     5-9    19.4981  21.3916  22.6219   17.5201  0.730602  0.770962  0.849598  
     50-59   22.991  9.27075  10.3208  0.400686   9.81043   10.2884   11.2835  
     60-69   22.991  9.27075  10.3208  0.400686   9.81043   10.2884   11.2835  
     70+     22.991  9.27075  10.3208  0.400686   9.81043   10.2884   11.2835  

[180 rows x 180 columns]

In [60]:
full_home_mtx.unstack().reorder_levels(['area','age']).sort_index().to_csv('../Data/base_contact_mtx/full_home_mtx.csv')

# $$\beta_{home}$$

In [61]:
# Calculating beta home
beta_home = 0.8 * full_home_mtx.unstack().reorder_levels(['area','age']).sort_index().sum(axis=1)/9

In [62]:
# Save beta_home
with open('../Data/demograph/beta_home.pickle', 'wb') as handle:
    pickle.dump(beta_home.values, handle, protocol=pickle.HIGHEST_PROTOCOL)